In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T

In [2]:
spark  = (
    SparkSession.builder.appName("Recipes ML model - Are you a dessert?")
    .config("spark.driver.memory", "8g")
    .getOrCreate()
)

In [3]:
food = spark.read.csv(
    "../Data/recipes/epi_r.csv", inferSchema=True, header=True
)

In [4]:
def sanitize_colum_names(name):
    """The function dropes all the speshel characters and whitespaces
    and replaces them with the alphanumeric characters or the underscore and
    we only keep alpha numeric characters"""
    
    answer = name
    for i, j in ((" ", "_"), ("-", "_"), ("/", "_"), ("&", "and")):
        answer = answer.replace(i, j)
    return "".join(
        char
        for char in answer
        if char.isalpha() or char.isdigit() or char == "_"
    )

In [5]:
food = food.toDF(*[sanitize_colum_names(name) for name in food.columns])

In [ ]:
import pandas as pd

pd.set_option("display.max_rows", 1000)

is_binary